Ingeniería de Software Basada en la Nube

#  Entrega 1
__________________

## Objetivo

Construir y ejecutar un sistema de software, con arquitectura monolítica, que reciba datos mediante una **interfaz gráfica** y los almacene en una **base de datos**.

## Requisitos

Tener [Docker](https://www.docker.com/) instalado. Si no, realizar la instalación de acuerdo a la siguiente información:

* Para sistemas operativos basados en **Linux**: [Docker Engine](https://docs.docker.com/engine/install/).
* Para sistema operativo **MacOS**: [Docker Desktop](https://www.docker.com/products/docker-desktop/).
* Para sistema operativo **Windows**: [Docker Desktop](https://www.docker.com/products/docker-desktop/).

## Laboratorio

**1.** Creación de instancia de base de datos de [MySQL](https://www.mysql.com/) en un contenedor Docker.

    - Nombre: isbn_db
    - Puerto: 3306
    - Contraseña de usuario administrador (root): 123

In [ ]:
docker run -p 3306:3306 --name isbn_db -e MYSQL_ROOT_PASSWORD=123 -d mysql:latest

**2.** Ejecutar instrucción dentro del contenedor creado para iniciar sesión en el cliente de MySQL:

*Al ejecutar la instrución se requerirá el ingreso de la contraseña del usuario root.*

In [ ]:
docker exec -it isbn_db mysql -u root -p

**3.** Crear base de datos **isbn**:

In [ ]:
CREATE DATABASE isbn;

**4.** Seleccionar base de datos **isbn**:

In [ ]:
USE isbn;

**5.** Crear tabla **task**:

In [ ]:
CREATE TABLE task (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    description TEXT
);

**6.** Consultar registros de la base de datos:

In [ ]:
SELECT * FROM task;

**7.** Crear un directorio llamado **isbn_mo**.

**8.** Dentro del directorio, crear los siguientes directorios:

    - models/
    - respositories/
    - services/
    - controllers/
    - templates/
    - static/

**9.** En el directorio **models**, crear archivo **task.py**:

In [ ]:
from flask_sqlalchemy import SQLAlchemy

db = SQLAlchemy()

class Task(db.Model):

    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)
    description = db.Column(db.Text)

**10.** En el directorio **repositories**, crear archivo **task_repository.py**:

In [ ]:
from models.task import Task, db

class TaskRepository:

    @staticmethod
    def create_task(name, description):
        task = Task(name=name, description=description)
        db.session.add(task)
        db.session.commit()
        return task

**11.** En el directorio **services**, crear archivo **task_service.py**:

In [ ]:
from repositories.task_repository import TaskRepository

class TaskService:

    @staticmethod
    def create_task(name, description):
        return TaskRepository.create_task(name, description)

**12.** En el directorio **controllers**, crear archivo **task_controller.py**:

In [ ]:
from flask import Blueprint, render_template, request, jsonify, redirect, url_for
from services.task_service import TaskService

task_blueprint = Blueprint('tasks', __name__)

@task_blueprint.route('/tasks', methods=['POST'])
def create_task():

    data = request.form
    name = data.get('name')
    description = data.get('description')

    if not name:
        return jsonify({'error': 'Name is required'}), 400

    TaskService.create_task(name, description)
    return redirect(url_for('tasks.index'))

@task_blueprint.route('/')
def index():
    return render_template('index.html')

**13.** En el directorio **templates**, crear archivo **index.html**:

In [ ]:
<!DOCTYPE html>
<html>
<head>
    <title>ISBN</title>
    <link rel="stylesheet" type="text/css" href="{{ url_for('static', filename='styles.css') }}">
</head>
<body>
    <h1>Tarea</h1>
    <form method="POST" action="/tasks">
        <label for="name">Nombre:</label><br>
        <input type="text" id="name" name="name" required><br>
        <label for="description">Descripción:</label><br>
        <textarea id="description" name="description"></textarea><br>
        <button type="submit">Crear</button>
    </form>
</body>
</html>

**14.** En el directorio **static**, crear archivo **styles.css**:

In [ ]:
body {
    font-family: Arial, sans-serif;
    margin: 500;
    padding: 0;
    background-color: #f4f4f4;
}

.container {
    width: 20%;
    margin: 0 auto;
    padding: 20px;
    background-color: #fff;
    border-radius: 5px;
    box-shadow: 0 2px 5px rgba(0,0,0,0.1);
}

h1 {
    text-align: center;
    margin-bottom: 20px;
}

label {
    font-weight: bold;
}

input[type="text"],
textarea {
    width: 100%;
    padding: 10px;
    margin-bottom: 10px;
    border: 1px solid #ccc;
    border-radius: 3px;
    box-sizing: border-box;
}

textarea {
    height: 100px;
}

button {
    display: block;
    width: 100%;
    padding: 10px;
    background-color: #007bff;
    color: #fff;
    border: none;
    border-radius: 3px;
    cursor: pointer;
}

button:hover {
    background-color: #0056b3;
}

**15.** En la raíz del proyecto (directorio **isbn_mo**), crear archivo **app.py**:

*Nota:* reemplazar la variable **IP_DOCKER_CONTAINER** (línea 6) por la IP del contenedor de la base de datos. Para ello ejecutar:

> docker ps -a

Copiar el **CONTAINER ID** del contenedor **isbn_db** y ejecutar:

> docker inspect CONTAINER ID

**IP_DOCKER_CONTAINER** = NetworkSettings > Networks > **IPAddress**

**app.py**:

In [ ]:
from flask import Flask
from models.task import db
from controllers.task_controller import *

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'mysql+mysqlconnector://root:123@IP_DOCKER_CONTAINER:3306/isbn'
db.init_app(app)

app.register_blueprint(task_blueprint)

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=4200)

**16.** En la raíz del proyecto (directorio **isbn_mo**), crear archivo **requirements.txt**:

In [ ]:
Flask==2.3.3
Flask-SQLAlchemy==3.1.1
mysql-connector-python==8.0.27

**17.** En la raíz del proyecto (directorio **isbn_mo**), crear archivo **Dockerfile**:

In [ ]:
FROM python

WORKDIR /app

COPY requirements.txt requirements.txt

RUN pip install -r requirements.txt

COPY . .

EXPOSE 4200

CMD ["python", "app.py"]

**18.** Creación de imagen Docker para la aplicación web (**isbn_mo**):

In [ ]:
docker build -t isbn_mo .

**19.** Ejecución del monolito en un contenedor Docker:

In [ ]:
docker run -p 4200:4200 isbn_mo

**20.** Acceder al sistema de sofwtare mediante un navegador web: [http://localhost:4200/](http://localhost:4200/).

**21.** Crear una tarea mediante la interfaz gráfica.

**22.** Verificar la creación de la tarea en la base de datos:

In [ ]:
SELECT * FROM task;

## Reto

Añadir una funcionalidad que permita **modificar** una tarea ya creada.

## Entregable

Archivo con nombre **cbse_l1_NombreApellido.pdf**, el cual debe contener:

- Nombre completo.
- Diagrama de flujo que describa el proceso realizado (para laboratorio y reto) y los resultados obtenidos.